# Create interfaces

This section is meant for the more advanced user. In it we will discuss how you can create your own interface, i.e. wrapping your own code, so that you can use it with Nipype.

In this notebook we will show you:

1. Example of an already implemented interface
2. What are the main parts of a Nipype interface?
3. How to wrap a CommandLine interface?
4. How to wrap a Python interface?
5. How to wrap a MATLAB interface?

But before we can start, let's recap again the difference between interfaces and workflows.

## Interfaces vs. Workflows

Interfaces are the building blocks that solve well-defined tasks. We solve more complex tasks by combining interfaces with workflows:

<table style="width: 100%; font-size: 14px;">
  <thead>
    <th style="text-align:left">Interfaces</th>
    <th style="text-align:left">Workflows</th>
  </thead>
  <tbody>
    <tr>
      <td style="text-align:left">Wrap *unitary* tasks</td>
      <td style="text-align:left">Wrap *meta*-tasks
        <li style="text-align:left">implemented with nipype interfaces wrapped inside ``Node`` objects</li>
        <li style="text-align:left">subworkflows can also be added to a workflow without any wrapping</li>
      </td>
    </tr>
    <tr>
      <td style="text-align:left">Keep track of the inputs and outputs, and check their expected types</td>
      <td style="text-align:left">Do not have inputs/outputs, but expose them from the interfaces wrapped inside</td>
    </tr>
    <tr>
      <td style="text-align:left">Do not cache results (unless you use [interface caching](advanced_interfaces_caching.ipynb))</td>
      <td style="text-align:left">Cache results</td>
    </tr>
    <tr>
      <td style="text-align:left">Run by a nipype plugin</td>
      <td style="text-align:left">Run by a nipype plugin</td>
    </tr>
  </tbody>
</table>

# Example of an already implemented interface

For this notebook, we'll work on the following T1-weighted dataset located in ``/data/ds000114/sub-01/ses-test/anat/sub-01_ses-test_T1w.nii.gz``:

In [ ]:
from nilearn.plotting import plot_anat
%matplotlib inline

In [ ]:
plot_anat('/data/ds000114/sub-01/ses-test/anat/sub-01_ses-test_T1w.nii.gz', dim=-1);

## Example of interface: FSL's `BET`

Nipype offers a series of Python interfaces to various external packages (e.g. FSL, SPM or FreeSurfer) even if they themselves are written in programming languages other than python. Such interfaces know what sort of options their corresponding tool has and how to execute it.

To illustrate why interfaces are so useful, let's have a look at the brain extraction algorithm [BET](http://fsl.fmrib.ox.ac.uk/fsl/fslwiki/BET) from FSL. Once in its original framework and once in the Nipype framework.

The tool can be run directly in a bash shell using the following command line:

In [ ]:
%%bash
bet /data/ds000114/sub-01/ses-test/anat/sub-01_ses-test_T1w.nii.gz \
    /data/ds000114/sub-01/ses-test/anat/sub-01_ses-test_T1w_bet.nii.gz

... which yields the following:

In [ ]:
plot_anat('/data/ds000114/sub-01/ses-test/anat/sub-01_ses-test_T1w_bet.nii.gz', dim=-1);

Using nipype, the equivalent is a bit more verbose:
  - line 1: The first line imports the interface
  - line 2: Then, the interface is instantiated. We provide here the input file.
  - line 3: Finally, we run the interface
  - line 4: The output file name can be automatically handled by nipype, and we will use that feature here

In [ ]:
from nipype.interfaces.fsl import BET
skullstrip = BET(in_file='/data/ds000114/sub-01/ses-test/anat/sub-01_ses-test_T1w.nii.gz')
res = skullstrip.run()
print(res.outputs.out_file)

Now we can verify that the result is exactly the same as before. Please note that, since we are using a Python environment, we use the result of the execution to point our ``plot_anat`` function to the output image of running BET:

In [ ]:
plot_anat(res.outputs.out_file, dim=-1);

# What are the main parts of a Nipype interface?

Nipype is designed to ease writing interfaces for new software. Nipype interfaces are designed with three elements that are intuitive:
  - A specification of inputs (or the ``InputSpec``)
  - A specification of outputs (or the ``OutputSpec``)
  - An interface *core* which implements the ``run()`` method we've seen before for BET, and which puts together inputs and outputs.

# The ``CommandLine`` interface

## A quick example

The easiest and quickest way to run any command line is the ``CommandLine`` interface, which has a very simple specification of inputs ready to use:

In [ ]:
from nipype.interfaces.base import CommandLine
CommandLine.help()

As a quick example, let's wrap bash's ``ls`` with Nipype:

In [ ]:
nipype_ls = CommandLine('ls', args='-lh', terminal_output='allatonce')

Now, we have a Python object ``nipype_ls`` that is a runnable nipype interface. After execution, Nipype interface returns a result object. We can retrieve the output of our ``ls`` invocation from the ``result.runtime`` property:

In [ ]:
result = nipype_ls.run()
print(result.runtime.stdout)

## Create your own `CommandLine` interface

Let's create a Nipype Interface for a very simple tool called ``antsTransformInfo`` from the [ANTs](http://stnava.github.io/ANTs/) package. This tool is so simple it does not even have a usage description for bash. Using it with a file, gives us the following result: 

In [ ]:
%%bash
antsTransformInfo /home/neuro/nipype_tutorial/notebooks/scripts/transform.tfm

### So let's plan our implementation:

  1. The command line name is ``antsTransformInfo``.
  2. It only accepts one text file (containing an ITK transform file) as input, and it is a positional argument.
  3. It prints out the properties of the transform in the input file. For the purpose of this notebook, we are only interested in extracting the translation values.
  
For the first item of this roadmap, we will just need to derive a new Python class from the ``nipype.interfaces.base.CommandLine`` base. To indicate the appropriate command line, we set the member ``_cmd``:

In [ ]:
class TransformInfo(CommandLine):
    _cmd = 'antsTransformInfo'

This is enough to have a nipype compatible interface for this tool:

In [ ]:
TransformInfo.help()

### Specifying the inputs

However, the ``args`` argument is too generic and does not deviate much from just running it in bash, or directly using ``subprocess.Popen``. Let's define the inputs specification for the interface, extending the ``nipype.interfaces.base.CommandLineInputSpec`` class.

The inputs are implemented using the Enthought traits package. For now, we'll use the ``File`` trait extension of nipype:

In [ ]:
from nipype.interfaces.base import CommandLineInputSpec, File

class TransformInfoInputSpec(CommandLineInputSpec):
    in_file = File(exists=True, mandatory=True, argstr='%s',
                   position=0, desc='the input transform file')

Some settings are done for this ``File`` object:
- ``exists=True`` indicates Nipype that the file must exist when it is set
- ``mandatory=True`` checks that this input was set before running because the program would crash otherwise
- ``argstr='%s'`` indicates how this input parameter should be formatted
- ``position=0`` indicates  that this is the first positional argument

We can now decorate our ``TransformInfo`` core class with its input, by setting the ``input_spec`` member:

In [ ]:
class TransformInfo(CommandLine):
    _cmd = 'antsTransformInfo'
    input_spec = TransformInfoInputSpec

Our interface now has one mandatory input, and inherits some optional inputs from the ``CommandLineInputSpec``:

In [ ]:
TransformInfo.help()

One interesting feature of the Nipype interface is that the underlying command line can be checked using the object property ``cmdline``. The command line can only be built when the mandatory inputs are set, so let's instantiate our new Interface for the first time, and check the underlying command line:

In [ ]:
my_info_interface = TransformInfo(in_file='/home/neuro/nipype_tutorial/notebooks/scripts/transform.tfm')
print(my_info_interface.cmdline)

Nipype will make sure that the parameters fulfill their prescribed attributes. For instance, ``in_file`` is mandatory. An error is issued if we build the command line or try to run this interface without it:

In [ ]:
try:
    TransformInfo().cmdline

except(ValueError) as err:
    print('It crashed with...')
    print("ValueError:", err)
else:
    raise

It will also complain if we try to set a non-existent file:

In [ ]:
try:
    my_info_interface.inputs.in_file = 'idontexist.tfm'

except(Exception) as err:
    print('It crashed with...')
    print("TraitError:", err)
else:
    raise

### Specifying the outputs
The outputs are defined in a similar way. Let's define a custom output for our interface which is a list of three float element. The output traits are derived from a simpler base class called ``TraitedSpec``. We also import the two data representations we need ``List`` and ``Float``:

In [ ]:
from nipype.interfaces.base import TraitedSpec, traits

class TransformInfoOutputSpec(TraitedSpec):
    translation = traits.List(traits.Float, desc='the translation component of the input transform')
    
class TransformInfo(CommandLine):
    _cmd = 'antsTransformInfo'
    input_spec = TransformInfoInputSpec
    output_spec = TransformInfoOutputSpec

And now, our new output is in place:

In [ ]:
TransformInfo.help()

### We are almost there - final needs
If we run the interface, we'll be able to see that this tool only writes some text to the standard output, but we just want to extract the ``Translation`` field and generate a Python object from it.

In [ ]:
my_info_interface = TransformInfo(in_file='/home/neuro/nipype_tutorial/notebooks/scripts/transform.tfm',
                                  terminal_output='allatonce')
result = my_info_interface.run()

In [ ]:
print(result.runtime.stdout)

We need to complete the functionality of the ``run()`` member of our interface to parse the standard output. This is done extending its ``_run_interface()`` member.

When we define outputs, generally they need to be explicitly wired in the ``_list_outputs()`` member of the core class. Let's see how we can *complete* those:

In [ ]:
class TransformInfo(CommandLine):
    _cmd = 'antsTransformInfo'
    input_spec = TransformInfoInputSpec
    output_spec = TransformInfoOutputSpec
            
    def _run_interface(self, runtime):
        import re
        
        # Run the command line as a natural CommandLine interface
        runtime = super(TransformInfo, self)._run_interface(runtime)

        # Search transform in the standard output
        expr_tra = re.compile('Translation:\s+\[(?P<translation>[0-9\.-]+,\s[0-9\.-]+,\s[0-9\.-]+)\]')
        trans = [float(v) for v in expr_tra.search(runtime.stdout).group('translation').split(', ')]
        
        # Save it for later use in _list_outputs
        setattr(self, '_result', trans)
        
        # Good to go
        return runtime
    
    def _list_outputs(self):
        
        # Get the attribute saved during _run_interface
        return {'translation': getattr(self, '_result')}

Let's run this interface (we set ``terminal_output='allatonce'`` to reduce the length of this manual, default would otherwise be `'stream'`):

In [ ]:
my_info_interface = TransformInfo(in_file='/home/neuro/nipype_tutorial/notebooks/scripts/transform.tfm',
                                  terminal_output='allatonce')
result = my_info_interface.run()

Now we can retrieve our outcome of interest as an output:

In [ ]:
result.outputs.translation

### Summary of a `CommandLine` interface

Now putting it all togehter, it looks as follows:

In [ ]:
from nipype.interfaces.base import (CommandLine, CommandLineInputSpec,
                                    TraitedSpec, traits, File)

class TransformInfoInputSpec(CommandLineInputSpec):
    in_file = File(exists=True, mandatory=True, argstr='%s', position=0,
                   desc='the input transform file')

class TransformInfoOutputSpec(TraitedSpec):
    translation = traits.List(traits.Float, desc='the translation component of the input transform')

class TransformInfo(CommandLine):
    _cmd = 'antsTransformInfo'
    input_spec = TransformInfoInputSpec
    output_spec = TransformInfoOutputSpec
            
    def _run_interface(self, runtime):
        import re
        
        # Run the command line as a natural CommandLine interface
        runtime = super(TransformInfo, self)._run_interface(runtime)

        # Search transform in the standard output
        expr_tra = re.compile('Translation:\s+\[(?P<translation>[0-9\.-]+,\s[0-9\.-]+,\s[0-9\.-]+)\]')
        trans = [float(v) for v in expr_tra.search(runtime.stdout).group('translation').split(', ')]
        
        # Save it for later use in _list_outputs
        setattr(self, '_result', trans)
        
        # Good to go
        return runtime
    
    def _list_outputs(self):
        
        # Get the attribute saved during _run_interface
        return {'translation': getattr(self, '_result')}

In [ ]:
my_info_interface = TransformInfo(in_file='/home/neuro/nipype_tutorial/notebooks/scripts/transform.tfm',
                                  terminal_output='allatonce')
result = my_info_interface.run()
result.outputs.translation

### Wrapping up - fast use case for simple `CommandLine` wrapper

For more standard neuroimaging software, generally we will just have to specify simple flags, i.e. input and output images and some additional parameters. If that is the case, then there is no need to extend the ``run()`` method.

Let's look at a quick, partial, implementation of FSL's BET:

In [ ]:
from nipype.interfaces.base import CommandLineInputSpec, File, TraitedSpec

class CustomBETInputSpec(CommandLineInputSpec):
    in_file = File(exists=True, mandatory=True, argstr='%s', position=0, desc='the input image')
    mask = traits.Bool(mandatory=False, argstr='-m', position=2, desc='create binary mask image')

    # Do not set exists=True for output files!
    out_file = File(mandatory=True, argstr='%s', position=1, desc='the output image')
    
class CustomBETOutputSpec(TraitedSpec):
    out_file = File(desc='the output image')
    mask_file = File(desc="path/name of binary brain mask (if generated)")
    
class CustomBET(CommandLine):
    _cmd = 'bet'
    input_spec = CustomBETInputSpec
    output_spec = CustomBETOutputSpec
    
    def _list_outputs(self):

        # Get the attribute saved during _run_interface
        return {'out_file': self.inputs.out_file,
                'mask_file': self.inputs.out_file.replace('brain', 'brain_mask')}

In [ ]:
my_custom_bet = CustomBET()
my_custom_bet.inputs.in_file = '/data/ds000114/sub-01/ses-test/anat/sub-01_ses-test_T1w.nii.gz'
my_custom_bet.inputs.out_file = 'sub-01_T1w_brain.nii.gz'
my_custom_bet.inputs.mask = True
result = my_custom_bet.run()

In [ ]:
plot_anat(result.outputs.out_file, dim=-1);

# Create your own `Python` interface

`CommandLine` interface is great, but my tool is already in Python - can I wrap it natively?

Sure. Let's solve the following problem: Let's say we have a Python function that takes an input image and a list of three translations (x, y, z) in mm, and then writes a resampled image after the translation has been applied:

In [ ]:
def translate_image(img, translation, out_file):

    import nibabel as nb
    import numpy as np
    from scipy.ndimage.interpolation import affine_transform
    
    # Load the data
    nii = nb.load(img)
    data = nii.get_data()
    
    # Create the transformation matrix
    matrix = np.eye(3)
    trans = (np.array(translation) / nii.header.get_zooms()[:3]) * np.array([1.0, -1.0, -1.0])
    
    # Apply the transformation matrix
    newdata = affine_transform(data, matrix=matrix, offset=trans)
    
    # Save the new data in a new NIfTI image
    nb.Nifti1Image(newdata, nii.affine, nii.header).to_filename(out_file)
    
    print('Translated file now is here: %s' % out_file)

Let's see how this function operates:

In [ ]:
orig_image = '/data/ds000114/sub-01/ses-test/anat/sub-01_ses-test_T1w.nii.gz'
translation = [20.0, -20.0, -20.0]
translated_image = 'translated.nii.gz'

# Let's run the translate_image function on our inputs
translate_image(orig_image,
                translation,
                translated_image)

Now that the function was executed, let's plot the original and the translated image.

In [ ]:
plot_anat(orig_image, dim=-1);

In [ ]:
plot_anat('translated.nii.gz', dim=-1);

Perfect, we see that the translation was applied.

## Quick approach - ``Function`` interface

Don't reinvent the wheel if it's not necessary. If like in this case, we have a well-defined function we want to run with Nipype, it is fairly easy to solve it with the ``Function`` interface:

In [ ]:
from nipype.interfaces.utility import Function

my_python_interface = Function(
    input_names=['img', 'translation', 'out_file'],
    output_names=['out_file'],
    function=translate_image
)

The arguments of ``translate_image`` should ideally be listed in the same order and with the same names as in the signature of the function. The same should be the case for the outputs. Finally, the ``Function`` interface takes a ``function`` input that is pointed to your python code.

***Note***: The inputs and outputs do not pass any kind of conformity checking: the function node will take any kind of data type for their inputs and outputs.

There are some other limitations to the ``Function`` interface when used inside workflows. Additionally, the function must be totally self-contained, since it will run with no global context. In practice, it means that **all the imported modules and variables must be defined within the context of the function**.

For more, check out the [Function Node](basic_function_nodes.ipynb) notebook.

Back to our `Function` interface. You can run it as any other interface object of Nipype:

In [ ]:
# Set inputs
my_python_interface.inputs.img = '/data/ds000114/sub-01/ses-test/anat/sub-01_ses-test_T1w.nii.gz'
my_python_interface.inputs.translation = [-35.0, 35.0, 35.0]
my_python_interface.inputs.out_file = 'translated_functioninterface.nii.gz'

In [ ]:
# Run the interface
result = my_python_interface.run()

In [ ]:
# Plot the result
plot_anat('translated_functioninterface.nii.gz', dim=-1);

## Complete approach - pure `Python` interface

Now, we face the problem of interfacing something different from a command line. Therefore, the ``CommandLine`` base class will not help us here. The specification of the inputs and outputs, though, will work the same way.

Let's start from that point on. Our Python function takes in three inputs: (1) the input image, (2) the translation and (3) an output image.

The specification of inputs and outputs must be familiar to you at this point. Please note that now, input specification is derived from ``BaseInterfaceInputSpec``, which is a bit thinner than ``CommandLineInputSpec``. The output specification can be derived from ``TraitedSpec`` as before:

In [ ]:
from nipype.interfaces.base import BaseInterfaceInputSpec, File, TraitedSpec

class TranslateImageInputSpec(BaseInterfaceInputSpec):
    in_file = File(exists=True, mandatory=True, desc='the input image')
    out_file = File(mandatory=True, desc='the output image') # Do not set exists=True !!
    translation = traits.List([50.0, 0.0, 0.0], traits.Float, usedefault=True,
                              desc='the translation component of the input transform')
    
class TranslateImageOutputSpec(TraitedSpec):
    out_file = File(desc='the output image')

Similarily to the change of base class for the input specification, the core of our new interface will derive from ``BaseInterface`` instead of ``CommandLineInterface``:

In [ ]:
from nipype.interfaces.base import BaseInterface

class TranslateImage(BaseInterface):
    input_spec = TranslateImageInputSpec
    output_spec = TranslateImageOutputSpec

At this point, we have defined a pure python interface but it is unable to do anything because we didn't implement a ``_run_interface()`` method yet.

In [ ]:
TranslateImage.help()

What happens if we try to run such an interface without specifying the `_run_interface()` function?

In [ ]:
will_fail_at_run = TranslateImage(
    in_file='/data/ds000114/sub-01/ses-test/anat/sub-01_ses-test_T1w.nii.gz',
    out_file='translated.nii.gz')

In [ ]:
try:
    result = will_fail_at_run.run()

except(NotImplementedError) as err:
    print('It crashed with...')
    print("NotImplementedError:", err)
else:
    raise

So, let's implement the missing part. As we would imagine, this needs to be very similar to what we did before with the ``TransformInfo`` interface:

In [ ]:
class TranslateImage(BaseInterface):
    input_spec = TranslateImageInputSpec
    output_spec = TranslateImageOutputSpec
    
    def _run_interface(self, runtime):
        
        # Call our python code here:
        translate_image(
            self.inputs.in_file,
            self.inputs.translation,
            self.inputs.out_file
        )
        
        # And we are done
        return runtime

If we run it know, our interface will get further:

In [ ]:
half_works = TranslateImage(
    in_file='/data/ds000114/sub-01/ses-test/anat/sub-01_ses-test_T1w.nii.gz',
    out_file='translated_nipype.nii.gz')

In [ ]:
try:
    result = half_works.run()

except(NotImplementedError) as err:
    print('It crashed with...')
    print("NotImplementedError:", err)
else:
    raise

... but still, it crashes becasue we haven't specified any ``_list_outputs()`` method. I.e. our python function is called, but the interface crashes when the execution arrives to retrieving the outputs.

Let's fix that:

In [ ]:
from nipype.interfaces.base import BaseInterfaceInputSpec, BaseInterface, File, TraitedSpec

class TranslateImageInputSpec(BaseInterfaceInputSpec):
    in_file = File(exists=True, mandatory=True, desc='the input image')
    out_file = File(mandatory=True, desc='the output image') # Do not set exists=True !!
    translation = traits.List([50.0, 0.0, 0.0], traits.Float, usedefault=True,
                              desc='the translation component of the input transform')
    
class TranslateImageOutputSpec(TraitedSpec):
    out_file = File(desc='the output image')

class TranslateImage(BaseInterface):
    input_spec = TranslateImageInputSpec
    output_spec = TranslateImageOutputSpec
    
    def _run_interface(self, runtime):

        # Call our python code here:
        translate_image(
            self.inputs.in_file,
            self.inputs.translation,
            self.inputs.out_file
        )
        # And we are done
        return runtime

    def _list_outputs(self):
        return {'out_file': self.inputs.out_file}

Now, we have everything together. So let's run it and visualize the output file.

In [ ]:
this_works = TranslateImage(
    in_file='/data/ds000114/sub-01/ses-test/anat/sub-01_ses-test_T1w.nii.gz',
    out_file='translated_nipype.nii.gz')

result = this_works.run()

In [ ]:
plot_anat(result.outputs.out_file, dim=-1);

# Create your own `MATLAB` interface

Last but not least, let's take a look at how we would create a `MATLAB` interface. For this purpose, let's say we want to run some matlab code that counts the number of voxels in an MRI image with intensity larger than zero. Such a value could give us an estimation of the brain volume (in voxels) of a skull-stripped image.

In `MATLAB`, our code looks as follows:

  ```
  load input_image.mat;
  total = sum(data(:) > 0)
  ```
  
The following example uses ``scipy.io.savemat`` to convert the input image to `MATLAB` format. Once the file is loaded we can quickly extract the estimated total volume.

***Note:*** For the purpose of this example, we will be using the freely available `MATLAB` alternative `Octave`. But the implementation of a `MATLAB` interface will be identical.

## Preparation

As before, we need to specify an `InputSpec` and an `OutputSpec` class. The input class will expect a `file` as an input and the `script` containing the code that we would like to run, and the output class will give us back the total `volume`.

In the context of a `MATLAB` interface, this is implemented as follows:

In [ ]:
from nipype.interfaces.base import (CommandLine, traits, TraitedSpec,
                                    BaseInterface, BaseInterfaceInputSpec, File)

class BrainVolumeMATLABInputSpec(BaseInterfaceInputSpec):
    in_file = File(exists=True, mandatory=True)
    script_file = File(exists=True, mandatory=True)
    
class BrainVolumeMATLABOutputSpec(TraitedSpec):
    volume = traits.Int(desc='brain volume')

class BrainVolumeMATLAB(BaseInterface):
    input_spec = BrainVolumeMATLABInputSpec
    output_spec = BrainVolumeMATLABOutputSpec

## Step by step implementation

Now, we have to specify what should happen, once the interface is run. As we said earlier, we want to:

1. load the image data and save it in a mat file
2. load the script
3. replace the put the relevant information into the script
4. run the script
5. extract the results

This all can be implemented with the following code:

In [ ]:
# Specify the interface inputs
in_file = '/data/ds000114/sub-01/ses-test/anat/sub-01_ses-test_T1w.nii.gz'
script_file = '/home/neuro/nipype_tutorial/notebooks/scripts/brainvolume.m'

In [ ]:
!cat scripts/brainvolume.m

In [ ]:
import re
import nibabel as nb
from scipy.io import savemat

# 1. save the image in matlab format as tmp_image.mat
tmp_image = 'tmp_image.mat'
data = nb.load(in_file).get_data()
savemat(tmp_image, {'data': data}, do_compression=False)

In [ ]:
# 2. load script
with open(script_file) as script_file:
    script_content = script_file.read()

In [ ]:
# 3. replace the input_image.mat file with the actual input of this interface
with open('newscript.m', 'w') as script_file:
    script_file.write(script_content.replace('input_image.mat', 'tmp_image.mat'))

In [ ]:
# 4. run the matlab script
mlab = CommandLine('octave', args='newscript.m', terminal_output='stream')
result = mlab.run()

In [ ]:
# 5. extract the volume estimation from the output
expr_tra = re.compile('total\ =\s+(?P<total>[0-9]+)')
volume = int(expr_tra.search(result.runtime.stdout).groupdict()['total'])
print(volume)

## Putting it all together

Now we just need to put this all together in the `_run_interface()` method and add a `_list_outputs()` function:

In [ ]:
from nipype.interfaces.base import (CommandLine, traits, TraitedSpec,
                                    BaseInterface, BaseInterfaceInputSpec, File)
import re
import nibabel as nb
from scipy.io import savemat

class BrainVolumeMATLABInputSpec(BaseInterfaceInputSpec):
    in_file = File(exists=True, mandatory=True)
    script_file = File(exists=True, mandatory=True)
    
class BrainVolumeMATLABOutputSpec(TraitedSpec):
    volume = traits.Int(desc='brain volume')

class BrainVolumeMATLAB(BaseInterface):
    input_spec = BrainVolumeMATLABInputSpec
    output_spec = BrainVolumeMATLABOutputSpec

    def _run_interface(self, runtime): 
        # Save the image in matlab format as tmp_image.mat
        tmp_image = 'tmp_image.mat'
        data = nb.load(self.inputs.in_file).get_data()
        savemat(tmp_image, {'data': data}, do_compression=False)
        
        # Load script
        with open(self.inputs.script_file) as script_file:
            script_content = script_file.read()
        
        # Replace the input_image.mat file for the actual input of this interface
        with open('newscript.m', 'w') as script_file:
            script_file.write(script_content.replace('input_image.mat', 'tmp_image.mat'))

        # Run a matlab command
        mlab = CommandLine('octave', args='newscript.m', terminal_output='stream')
        result = mlab.run()
        
        expr_tra = re.compile('total\ =\s+(?P<total>[0-9]+)')
        volume = int(expr_tra.search(result.runtime.stdout).groupdict()['total'])
        setattr(self, '_result', volume)
        return result.runtime

    def _list_outputs(self):
        outputs = self._outputs().get()
        outputs['volume'] = getattr(self, '_result')
        return outputs

Let's test it:

In [ ]:
matlab = BrainVolumeMATLAB(in_file='/data/ds000114/sub-01/ses-test/anat/sub-01_ses-test_T1w.nii.gz',
                           script_file='/home/neuro/nipype_tutorial/notebooks/scripts/brainvolume.m')
result = matlab.run()

In [ ]:
print(result.outputs)

We see in the example above that everything works fine. But now, let's say that we want to save the total brain volume to a file and give the location of this file back as an output. How would you do that?

## Exercise

Modify the `BrainVolumeMATLAB` interface so that it has one more **output** called ``out_file``, that points to a text file where we write the volume in voxels. The name of the ``out_file`` can be hard coded to ``volume.txt``.

In [ ]:
# Write your solution here

In [ ]:
from nipype.interfaces.base import (CommandLine, traits, TraitedSpec,
                                    BaseInterface, BaseInterfaceInputSpec, File)
import os
import re
import nibabel as nb
from scipy.io import savemat

class BrainVolumeMATLABInputSpec(BaseInterfaceInputSpec):
    in_file = File(exists=True, mandatory=True)
    script_file = File(exists=True, mandatory=True)
    
class BrainVolumeMATLABOutputSpec(TraitedSpec):
    volume = traits.Int(desc='brain volume')
    out_file = File(desc='output file containing total brain volume')  # This line was added

class BrainVolumeMATLAB(BaseInterface):
    input_spec = BrainVolumeMATLABInputSpec
    output_spec = BrainVolumeMATLABOutputSpec

    def _run_interface(self, runtime): 
        # Save the image in matlab format as tmp_image.mat
        tmp_image = 'tmp_image.mat'
        data = nb.load(self.inputs.in_file).get_data()
        savemat(tmp_image, {'data': data}, do_compression=False)
        
        # Load script
        with open(self.inputs.script_file) as script_file:
            script_content = script_file.read()
        
        # Replace the input_image.mat file for the actual input of this interface
        with open('newscript.m', 'w') as script_file:
            script_file.write(script_content.replace('input_image.mat', 'tmp_image.mat'))

        # Run a matlab command
        mlab = CommandLine('octave', args='newscript.m', terminal_output='stream')
        result = mlab.run()
        
        expr_tra = re.compile('total\ =\s+(?P<total>[0-9]+)')
        volume = int(expr_tra.search(result.runtime.stdout).groupdict()['total'])
        setattr(self, '_result', volume)
        
        # Write total brain volume into a file
        out_fname = os.path.abspath('volume.txt')
        setattr(self, '_out_file', out_fname)
        with open('volume.txt', 'w') as out_file:
            out_file.write('%d' %volume)
        
        return result.runtime

    def _list_outputs(self):
        outputs = self._outputs().get()
        outputs['volume'] = getattr(self, '_result')
        outputs['out_file'] = getattr(self, '_out_file')
        return outputs

Now, let's test if it works.

In [ ]:
matlab = BrainVolumeMATLAB(in_file='/data/ds000114/sub-01/ses-test/anat/sub-01_ses-test_T1w.nii.gz',
                           script_file='/home/neuro/nipype_tutorial/notebooks/scripts/brainvolume.m')
result = matlab.run()

No errors, perfect. Did we get the right file?

In [ ]:
print(result.outputs.out_file)

And what about the content of this file?

In [ ]:
!cat volume.txt